In [23]:
!pip install simplet5
!pip install scikit-learn
import torch
import pandas as pd
from tqdm import tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Preprocessing

In [5]:
# Load the training dataset
df = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv", encoding='utf-8')
df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [6]:
# Drop id column and reset index of training data
df = df.drop(['id'], axis=1)
df = df.reset_index(drop=True)

In [7]:
# Renaming the columns as T5 identifies them with specific names
df = df.rename(columns={"highlights":"source_text", "article":"target_text"})
df = df[['source_text', 'target_text']]

# Adding a prefix named summarize as T5 expects a prefix of the task to be accomplished
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,"summarize: Bishop John Folda, of North Dakota,...",By . Associated Press . PUBLISHED: . 14:11 EST...
1,summarize: Criminal complaint: Cop used his ro...,(CNN) -- Ralph Mata was an internal affairs li...
2,"summarize: Craig Eccleston-Todd, 27, had drunk...",A drunk driver who killed a young woman in a h...
3,summarize: Nina dos Santos says Europe must be...,(CNN) -- With a breezy sweep of his pen Presid...
4,summarize: Fleetwood top of League One after 2...,Fleetwood are the only team still to have a 10...
...,...,...
287108,summarize: Chelsea Clinton said question of ru...,By . James Rush . Former first daughter Chelse...
287109,"summarize: Vanilla Ice, 47 - real name Robert ...",An apologetic Vanilla Ice has given his first ...
287110,summarize: America's most lethal sniper made c...,America's most lethal sniper claimed he wished...
287111,summarize: A swarm of more than one million ha...,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar..."


In [8]:
#Splitting the data into training and testing data
#I am using 90% data for training and 10 % for test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)
train_df.shape, test_df.shape

((258401, 2), (28712, 2))

# Creating and Training the model

In [9]:
# Using the pretrained T5 model this time as my last attempt to summarize using LSTM failed
from simplet5 import SimpleT5
model = SimpleT5() 

In [10]:
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [11]:
# Training the model for 2 epochs
model.train(train_df=train_df[:20000], 
            eval_df=test_df[:300],  
            source_max_token_len=300, 
            target_max_token_len=100, 
            batch_size=16, max_epochs=2, use_gpu=True, 
            early_stopping_patience_epochs=2)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
filepath = "model.pt"

# saving the model
torch.save(model, filepath)

# Generating the results

In [15]:
# Loading the moadel
model1 = torch.load('model.pt', map_location=torch.device('cuda'))
model1.device= torch.device("cuda")

In [16]:
# Giving a random news article dated 28-04-2023 as input
text_input = """A BJP delegation on Friday met the Election Commission and requested the poll panel to register a criminal case against Congress president Mallikarjun Kharge and bar him from campaign in the Karnataka assembly polls over his \"poisonous snake\" barb at Prime Minister Narendra Modi.Union minister Bhupender Yadav, who led the BJP delegation, told reporters that Kharge's comment was not merely a slip of the tongue but part of the Congress\' \"hate politics\".BJP general secretary Tarun Chugh said Kharge is a \"habitual offender\".Yadav said the Congress has a history of launching personal attacks on Modi.The BJP has demanded registration of an FIR under IPC Sections 499 and 500, which deal with defamation, and Section 504, which deals with the offence of deliberate insults and provocation.The delegation also included BJP chief spokesperson Anil Baluni and party leader Om Pathak.Addressing a campaign rally for the May 10 Karnataka assembly polls, Kharge likened Modi to a poisonous snake. As a row erupted, he later sought to clarify that his remarks were not aimed at the prime minister but at the ruling BJP."""
text_to_summarize = """summarize: """ + text_input

In [17]:
# Generating the output
predicted_output = model1.predict(text_to_summarize)
print(predicted_output)

['(CNN) -- A BJP delegation on Friday asked the Election Commission to register a criminal case against Congress president Mallikarjun Kharge and bar him from campaigning in the Karnataka assembly polls over his "poisonous snake" barb at Prime Minister Narendra Modi. The BJP\'s chief spokesperson, Anil Baluni, met with the Election Commission and requested the poll panel to register a criminal case against Kharge']


**This model has much better text summarisation capability than the seq2seq Bidirectional LSTM model**

# Calculating rouge score

In [49]:
#Taking first 20 abstract summaries as reference
summary_reference=df['target_text'][:20]
print(summary_reference)

0     By . Associated Press . PUBLISHED: . 14:11 EST...
1     (CNN) -- Ralph Mata was an internal affairs li...
2     A drunk driver who killed a young woman in a h...
3     (CNN) -- With a breezy sweep of his pen Presid...
4     Fleetwood are the only team still to have a 10...
5     He's been accused of making many a fashion fau...
6     By . Daily Mail Reporter . PUBLISHED: . 01:15 ...
7     By . Daily Mail Reporter . This is the moment ...
8     There are a number of job descriptions waiting...
9     Canberra, Australia (CNN) -- At first glance, ...
10    By . Ellie Zolfagharifard . Take a look at a m...
11    Two lawyers representing a woman who . claims ...
12    It's the moment every pet owner dreads - when ...
13    Louis van Gaal said he had no option but to su...
14    (CNN) -- One can hardly read the news these da...
15    For most people, it has become a travel essent...
16    By . Margot Peppers . Nigerian and Cameroonian...
17    Britain and the West must brace themselves

In [27]:
# Generating summaries using the model for first 20 stories
model_summaries=[]
for i in tqdm(range(20)):
    model_summary = model1.predict(df['source_text'][i])
    model_summaries.append(model_summary)

100%|██████████| 20/20 [00:33<00:00,  1.65s/it]


In [51]:
model_summaries = [str(summary) for summary in model_summaries]
print(model_summaries)

["['By. Daily Mail Reporter. PUBLISHED:. 07:11 EST, 19 October 2013. |. UPDATED:. 08:09 EST, 19 October 2013. A North Dakota man has been diagnosed with an infection that could have spread to his church members after contaminated food in Italy. Bishop John Folda, of North Dakota, is taking time off after being diagnosed with the illness. Church members in Fargo']", "['(CNN) -- A sex offender was arrested Friday in New York on suspicion of helping cocaine traffickers, a criminal complaint said. Ralph Mata, an internal affairs lieutenant, is accused of using his role as a drug dealer to help the group get guns and arrange to pay two assassins in a murder plot, the complaint said. He also arranged to pay the two assassins in a murder plot, the']", '["By. Daily Mail Reporter. PUBLISHED:. 07:10 EST, 14 October 2013. |. UPDATED:. 10:15 EST, 14 October 2013. A man who crashed his car head-on into a woman\'s car was found guilty at Portsmouth Crown Court of causing death by dangerous driving. 

In [29]:
!pip install rouge
from rouge import Rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
# Generating rouge score
rouge = Rouge() #Initialising the rouge function
rouge.get_scores(model_summaries, summary_reference, avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.10752574906332839,
  'p': 0.6039516507677088,
  'f': 0.17812368121941233},
 'rouge-2': {'r': 0.03330319665432749,
  'p': 0.27687537197000733,
  'f': 0.05760388228402452},
 'rouge-l': {'r': 0.10301351388435849,
  'p': 0.5785692318176411,
  'f': 0.17061348921457975}}